In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [ ]:
# sklearn 1.0 버전 부터 boston 주택가격 데이터셋이 삭제되었다.
# 다음과 같이 boston 데이터셋을 불러온다.
from sklearn.datasets import fetch_openml
boston = fetch_openml(name='boston')

df = pd.DataFrame(boston.data, columns=boston.feature_names)
df["TARGET"] = boston.target

In [6]:
# 표준 스케일링(standard scaling)을 통해 입력 값을 정규화한다.
scaler = StandardScaler()
scaler.fit(df.values[:, :-1])
df.values[:, :-1] = scaler.transform(df.values[:, :-1]).round(4)

df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0,0.573,6.030,80.8,2.5050,1,273.0,21.0,396.90,7.88,11.9


In [ ]:
# 정규화를 하는 이유
# 적절한 정규화는 신경망의 최적화를 수월하게 만들 수 있다.
# 정규화를 적용하기 전, 데이터셋 분포의 특징을 파악하고 어떤 정규화 방법이 가장 어울릴지 결정한다.
# 위 예제에선, 보스턴 주택가격 데이터셋의 각 열이 정규분포(normal distribution)을 따른다고 가정하고, 표준 스케일링을 적용한다.